In [33]:
from datetime import datetime
import pickle
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as colors
import plotly.io as pio
plt.style.use('ggplot')

# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set()
sns.set_style('whitegrid', {'font.sans-serif': ['simhei','FangSong']})

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

In [3]:
sph = pd.read_csv('./data/sph6004_assignment1_data.csv', index_col='id')

In [4]:
sph.head()

,aki,gender,admission_age,race,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,sbp_mean,dbp_min,dbp_max,dbp_mean,mbp_min,mbp_max,mbp_mean,resp_rate_min,resp_rate_max,resp_rate_mean,temperature_min,temperature_max,temperature_mean,spo2_min,spo2_max,spo2_mean,glucose_min,glucose_max,glucose_mean,lactate_min,lactate_max,ph_min,ph_max,so2_min,so2_max,po2_min,po2_max,pco2_min,pco2_max,aado2_min,aado2_max,aado2_calc_min,aado2_calc_max,pao2fio2ratio_min,pao2fio2ratio_max,baseexcess_min,baseexcess_max,bicarbonate_min,bicarbonate_max,totalco2_min,totalco2_max,hematocrit_min,hematocrit_max,hemoglobin_min,hemoglobin_max,carboxyhemoglobin_min,carboxyhemoglobin_max,methemoglobin_min,methemoglobin_max,temperature_min.1,temperature_max.1,chloride_min,chloride_max,calcium_min,calcium_max,glucose_min.1,glucose_max.1,potassium_min,potassium_max,sodium_min,sodium_max,hematocrit_min.1,hematocrit_max.1,hemoglobin_min.1,hemoglobin_max.1,platelets_min,platelets_max,wbc_min,wbc_max,albumin_min,albumin_max,globulin_min,globulin_max,total_protein_min,total_protein_max,aniongap_min,aniongap_max,bicarbonate_min.1,bicarbonate_max.1,bun_min,bun_max,calcium_min.1,calcium_max.1,chloride_min.1,chloride_max.1,glucose_min.2,glucose_max.2,sodium_min.1,sodium_max.1,potassium_min.1,potassium_max.1,abs_basophils_min,abs_basophils_max,abs_eosinophils_min,abs_eosinophils_max,abs_lymphocytes_min,abs_lymphocytes_max,abs_monocytes_min,abs_monocytes_max,abs_neutrophils_min,abs_neutrophils_max,atyps_min,atyps_max,bands_min,bands_max,imm_granulocytes_min,imm_granulocytes_max,metas_min,metas_max,nrbc_min,nrbc_max,d_dimer_min,d_dimer_max,fibrinogen_min,fibrinogen_max,thrombin_min,thrombin_max,inr_min,inr_max,pt_min,pt_max,ptt_min,ptt_max,alt_min,alt_max,alp_min,alp_max,ast_min,ast_max,amylase_min,amylase_max,bilirubin_total_min,bilirubin_total_max,bilirubin_direct_min,bilirubin_direct_max,bilirubin_indirect_min,bilirubin_indirect_max,ck_cpk_min,ck_cpk_max,ck_mb_min,ck_mb_max,ggt_min,ggt_max,ld_ldh_min,ld_ldh_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,height,weight_admit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36570066,3,F,79.953141,BLACK/AFRICAN AMERICAN,96.0,104.0,100.083333,103.0,126.0,116.136364,40.0,58.0,47.863636,56.0,69.0,63.090909,13.0,24.0,16.833333,35.89,36.39,36.222000,96.0,100.0,98.916667,76.0,211.0,146.2,2.3,2.3,7.40,7.40,98.0,98.0,101.0,101.0,47.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,30.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,NaN,NaN,NaN,NaN,5.2,5.2,129.0,129.0,27.8,29.2,9.2,9.8,304.0,333.0,11.0,14.3,2.6,2.9,NaN,NaN,NaN,NaN,15.0,20.0,23.0,28.0,47.0,52.0,8.1,9.0,93.0,101.0,76.0,167.0,131.0,138.0,4.8,6.7,0.0000,0.0000,0.0000,0.3300,0.2200,0.8220,0.5480,0.8800,7.8100,10.5490,0.0,0.0,5.0,11.0,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1,1.1,11.9,12.0,28.6,29.5,103.0,133.0,992.0,1199.0,106.0,196.0,NaN,NaN,6.8,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.0,318.0,15.0,6.0,5.0,4.0,0.0,157.0,110.0
39307659,0,F,78.194169,WHITE - RUSSIAN,72.0,134.0,97.263158,97.0,127.0,109.833333,56.0,89.0,70.166667,65.0,96.0,79.555556,17.0,38.0,29.611111,36.28,37.00,36.558000,91.0,98.0,95.444444,127.0,132.0,129.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.3,39.5,12.1,13.0,199.0,247.0,12.0,14.9,3.7,3.9,NaN,NaN,NaN,NaN,13.0,22.0,19.0,29.0,23.0,30.0,7.9,8.4,89.0,93.0,127.0,143.0,126.0,132.0,2.9,4.5,0.0294,0.0294,0.0147,0.0147,1.5141,1.5141,0.5586,0.5586,12.5832,12.5832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9,2.3,20.0,24.7,28.4,150.0,272.0,384.0,282.0,331.0,183.0,356.0,NaN,NaN,1.9,2.9,NaN,NaN,NaN,NaN,175.0,175.0,8.0,8.0,NaN,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,82.0
38743306,2,F,65.602396,WHITE,60.0,97.0,84.166667,95.0,143.0,112.153846,56.0,99.0,73.307692,69

In [5]:
sph.shape

(50920, 161)

In [6]:
from pandas_profiling import ProfileReport

In [7]:
profile = ProfileReport(sph, title="Pandas Profiling Report", explorative=True)

In [28]:
profile.to_file("./data/Pandas-Profiling-Report.html")

Export report to file: 100%|██████████| 1/1 [00:05<00:00,  5.97s/it]


In [8]:
missing_values_count = sph.isnull().sum()
missing_values_percentage = (missing_values_count / len(sph)) * 100

sorted_missing_values = missing_values_percentage.sort_values(ascending=False)
print(sorted_missing_values)

thrombin_min              99.821288
thrombin_max              99.821288
d_dimer_min               99.785939
d_dimer_max               99.785939
ggt_max                   99.073056
ggt_min                   99.073056
globulin_min              98.654753
globulin_max              98.654753
bicarbonate_min           98.332679
bicarbonate_max           98.332679
methemoglobin_max         97.839749
methemoglobin_min         97.839749
total_protein_max         97.723881
total_protein_min         97.723881
carboxyhemoglobin_min     97.651218
carboxyhemoglobin_max     97.651218
bilirubin_indirect_max    95.881775
bilirubin_indirect_min    95.881775
nrbc_min                  95.866064
nrbc_max                  95.866064
bilirubin_direct_max      95.695208
bilirubin_direct_min      95.695208
amylase_min               93.493716
amylase_max               93.493716
aado2_min                 93.142184
aado2_max                 93.142184
atyps_max                 88.421053
atyps_min                 88

In [9]:
len(sorted_missing_values[sorted_missing_values > 30])

97

In [10]:
sorted_missing_values[sorted_missing_values < 30].index

Index(['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'r

缺失值处理策略：
1. 观察发现缺失值较多，特征中缺失值比例达到 30% 以上的直接删除掉
    缺失值过多如果进行插补可能会带入较大的噪音，对预测结果造成不良影响，当前特征数量较多，所以选择直接删除掉。
2. 删除之后其他特征的缺失比例都在 10% 及以下，通过随机森林算法插补
    首先挑选出 5% 不含缺失值的数据作为测试集，这部分数据不需要填补，可以避免引入过拟合风险
    对剩余数据从缺失值最少的特征开始，将其作为预测值，其他特征缺失部分先以 0 填补加上原预测值作为 X，然后使用随机森林模型做训练进行预测；之后依次处理其他缺失特征
    


In [11]:
sph = sph.loc[:, ['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'race', 'gender',
       'admission_age', 'aki']]

In [12]:
sph.head()

,ptt_max,ptt_min,inr_min,inr_max,pt_min,pt_max,calcium_max.1,calcium_min.1,temperature_max,temperature_min,temperature_mean,glucose_min,glucose_max,glucose_mean,weight_admit,glucose_max.2,glucose_min.2,aniongap_max,aniongap_min,gcs_motor,potassium_min.1,potassium_max.1,bicarbonate_max.1,bicarbonate_min.1,sodium_min.1,sodium_max.1,chloride_min.1,chloride_max.1,gcs_verbal,hemoglobin_max.1,hemoglobin_min.1,wbc_max,wbc_min,platelets_max,platelets_min,hematocrit_max.1,hematocrit_min.1,bun_max,bun_min,gcs_eyes,gcs_min,gcs_unable,resp_rate_mean,resp_rate_max,resp_rate_min,spo2_max,spo2_mean,spo2_min,mbp_min,mbp_max,mbp_mean,dbp_max,dbp_min,sbp_mean,sbp_max,sbp_min,dbp_mean,heart_rate_min,heart_rate_max,heart_rate_mean,race,gender,admission_age,aki
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36570066,29.5,28.6,1.1,1.1,11.9,12.0,9.0,8.1,36.39,35.89,36.222000,76.0,211.0,146.2,110.0,167.0,76.0,20.0,15.0,6.0,4.8,6.7,28.0,23.0,131.0,138.0,93.0,101.0,5.0,9.8,9.2,14.3,11.0,333.0,304.0,29.2,27.8,52.0,47.0,4.0,15.0,0.0,16.833333,24.0,13.0,100.0,98.916667,96.0,56.0,69.0,63.090909,58.0,40.0,116.136364,126.0,103.0,47.863636,96.0,104.0,100.083333,BLACK/AFRICAN AMERICAN,F,79.953141,3
39307659,150.0,28.4,1.9,2.3,20.0,24.7,8.4,7.9,37.00,36.28,36.558000,127.0,132.0,129.5,82.0,143.0,127.0,22.0,13.0,6.0,2.9,4.5,29.0,19.0,126.0,132.0,89.0,93.0,5.0,13.0,12.1,14.9,12.0,247.0,199.0,39.5,36.3,30.0,23.0,4.0,15.0,0.0,29.611111,38.0,17.0,98.0,95.444444,91.0,65.0,96.0,79.555556,89.0,56.0,109.833333,127.0,97.0,70.166667,72.0,134.0,97.263158,WHITE - RUSSIAN,F,78.194169,0
38743306,26.0,26.0,1.1,1.1,12.1,12.1,8.7,8.7,37.00,36.67,36.805000,207.0,305.0,254.0,62.1,266.0,266.0,12.0,12.0,6.0,3.8,3.8,27.0,27.0,137.0,137.0,102.0,102.0,5.0,9.5,9.5,7.5,7.5,346.0,346.0,31.1,31.1,8.0,8.0,4.0,15.0,0.0,16.000000,19.0,14.0,100.0,97.166667,94.0,69.0,108.0,85.692308,99.0,56.0,112.153846,143.0,95.0,73.307692,60.0,97.0,84.166667,WHITE,F,65.602396,2
32339865,26.1,26.1,1.2,1.2,12.8,12.8,9.5,8.3,37.28,36.83,37.087143,111.0,120.0,117.0,113.1,136.0,120.0,11.0,10.0,1.0,4.3,4.5,39.0,36.0,140.0,141.0,92.0,93.0,0.0,18.6,16.1,7.2,5.1,178.0,146.0,62.8,56.5,39.0,39.0,1.0,15.0,1.0,22.518519,31.0,14.0,100.0,94.800000,89.0,71.0,109.0,96.720000,94.0,60.0,138.160000,150.0,113.0,80.200000,59.0,87.0,71.461538,UNKNOWN,F,64.906629,2
35526987,39.4,27.7,1.0,1.1,10.4,11.9,8.0,7.5,38.33,36.78,37.158750,158.0,213.0,187.0,97.4,196.0,132.0,16.0,13.0,NaN,4.2,4.5,22.0,18.0,137.0,137.0,103.0,109.0,0.0,17.1,14.0,21.1,14.6,208.0,147.0,54.3,44.0,23.0,22.0,1.0,15.0,1.0,24.230159,32.0,18.0,99.0,95.951613,90.0,56.0,100.0,70.475806,95.0,47.0,97.672131,127.0,81.0,61.827869,57.0,100.0,82.387097,WHITE,M,57.438861,2


In [13]:
sph.shape

(50920, 64)

In [15]:
profile = ProfileReport(sph, title="Pandas Profiling Report 2", explorative=True)
profile.to_file("./data/Pandas-Profiling-Report2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Export report to file: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


截取 5% 的无缺失值部分数据作为测试集

In [69]:
label_encoder = LabelEncoder()
sph["race"] = label_encoder.fit_transform(sph["race"])
print(label_encoder.classes_)

['AMERICAN INDIAN/ALASKA NATIVE' 'ASIAN' 'ASIAN - ASIAN INDIAN'
 'ASIAN - CHINESE' 'ASIAN - KOREAN' 'ASIAN - SOUTH EAST ASIAN'
 'BLACK/AFRICAN' 'BLACK/AFRICAN AMERICAN' 'BLACK/CAPE VERDEAN'
 'BLACK/CARIBBEAN ISLAND' 'HISPANIC OR LATINO'
 'HISPANIC/LATINO - CENTRAL AMERICAN' 'HISPANIC/LATINO - COLUMBIAN'
 'HISPANIC/LATINO - CUBAN' 'HISPANIC/LATINO - DOMINICAN'
 'HISPANIC/LATINO - GUATEMALAN' 'HISPANIC/LATINO - HONDURAN'
 'HISPANIC/LATINO - MEXICAN' 'HISPANIC/LATINO - PUERTO RICAN'
 'HISPANIC/LATINO - SALVADORAN' 'MULTIPLE RACE/ETHNICITY'
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' 'OTHER'
 'PATIENT DECLINED TO ANSWER' 'PORTUGUESE' 'SOUTH AMERICAN'
 'UNABLE TO OBTAIN' 'UNKNOWN' 'WHITE' 'WHITE - BRAZILIAN'
 'WHITE - EASTERN EUROPEAN' 'WHITE - OTHER EUROPEAN' 'WHITE - RUSSIAN']


In [70]:
label_encoder = LabelEncoder()
sph["gender"] = label_encoder.fit_transform(sph["gender"])
print(label_encoder.classes_)

['F' 'M']


In [16]:
sph_clean = sph.dropna()
test_set = sph_clean.sample(n=2546, random_state=20)

In [17]:
test_set.shape

(2546, 64)

In [18]:
sph_clean.shape

(38300, 64)

In [19]:
test_set.index

Int64Index([36186843, 31772481, 39391355, 35191106, 39739359, 35881376,
            35357825, 37574767, 37307218, 36557827,
            ...
            33640950, 38442425, 34659358, 38800171, 31315722, 31065288,
            33221394, 32348707, 35423127, 31779767],
           dtype='int64', name='id', length=2546)

In [20]:
sph.shape

(50920, 64)

In [21]:
train_set = sph.drop(test_set.index)

In [22]:
train_set.shape

(48374, 64)

In [23]:
train_set.head()

,ptt_max,ptt_min,inr_min,inr_max,pt_min,pt_max,calcium_max.1,calcium_min.1,temperature_max,temperature_min,temperature_mean,glucose_min,glucose_max,glucose_mean,weight_admit,glucose_max.2,glucose_min.2,aniongap_max,aniongap_min,gcs_motor,potassium_min.1,potassium_max.1,bicarbonate_max.1,bicarbonate_min.1,sodium_min.1,sodium_max.1,chloride_min.1,chloride_max.1,gcs_verbal,hemoglobin_max.1,hemoglobin_min.1,wbc_max,wbc_min,platelets_max,platelets_min,hematocrit_max.1,hematocrit_min.1,bun_max,bun_min,gcs_eyes,gcs_min,gcs_unable,resp_rate_mean,resp_rate_max,resp_rate_min,spo2_max,spo2_mean,spo2_min,mbp_min,mbp_max,mbp_mean,dbp_max,dbp_min,sbp_mean,sbp_max,sbp_min,dbp_mean,heart_rate_min,heart_rate_max,heart_rate_mean,race,gender,admission_age,aki
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36570066,29.5,28.6,1.1,1.1,11.9,12.0,9.0,8.1,36.39,35.89,36.222000,76.0,211.0,146.2,110.0,167.0,76.0,20.0,15.0,6.0,4.8,6.7,28.0,23.0,131.0,138.0,93.0,101.0,5.0,9.8,9.2,14.3,11.0,333.0,304.0,29.2,27.8,52.0,47.0,4.0,15.0,0.0,16.833333,24.0,13.0,100.0,98.916667,96.0,56.0,69.0,63.090909,58.0,40.0,116.136364,126.0,103.0,47.863636,96.0,104.0,100.083333,BLACK/AFRICAN AMERICAN,F,79.953141,3
39307659,150.0,28.4,1.9,2.3,20.0,24.7,8.4,7.9,37.00,36.28,36.558000,127.0,132.0,129.5,82.0,143.0,127.0,22.0,13.0,6.0,2.9,4.5,29.0,19.0,126.0,132.0,89.0,93.0,5.0,13.0,12.1,14.9,12.0,247.0,199.0,39.5,36.3,30.0,23.0,4.0,15.0,0.0,29.611111,38.0,17.0,98.0,95.444444,91.0,65.0,96.0,79.555556,89.0,56.0,109.833333,127.0,97.0,70.166667,72.0,134.0,97.263158,WHITE - RUSSIAN,F,78.194169,0
38743306,26.0,26.0,1.1,1.1,12.1,12.1,8.7,8.7,37.00,36.67,36.805000,207.0,305.0,254.0,62.1,266.0,266.0,12.0,12.0,6.0,3.8,3.8,27.0,27.0,137.0,137.0,102.0,102.0,5.0,9.5,9.5,7.5,7.5,346.0,346.0,31.1,31.1,8.0,8.0,4.0,15.0,0.0,16.000000,19.0,14.0,100.0,97.166667,94.0,69.0,108.0,85.692308,99.0,56.0,112.153846,143.0,95.0,73.307692,60.0,97.0,84.166667,WHITE,F,65.602396,2
32339865,26.1,26.1,1.2,1.2,12.8,12.8,9.5,8.3,37.28,36.83,37.087143,111.0,120.0,117.0,113.1,136.0,120.0,11.0,10.0,1.0,4.3,4.5,39.0,36.0,140.0,141.0,92.0,93.0,0.0,18.6,16.1,7.2,5.1,178.0,146.0,62.8,56.5,39.0,39.0,1.0,15.0,1.0,22.518519,31.0,14.0,100.0,94.800000,89.0,71.0,109.0,96.720000,94.0,60.0,138.160000,150.0,113.0,80.200000,59.0,87.0,71.461538,UNKNOWN,F,64.906629,2
35526987,39.4,27.7,1.0,1.1,10.4,11.9,8.0,7.5,38.33,36.78,37.158750,158.0,213.0,187.0,97.4,196.0,132.0,16.0,13.0,NaN,4.2,4.5,22.0,18.0,137.0,137.0,103.0,109.0,0.0,17.1,14.0,21.1,14.6,208.0,147.0,54.3,44.0,23.0,22.0,1.0,15.0,1.0,24.230159,32.0,18.0,99.0,95.951613,90.0,56.0,100.0,70.475806,95.0,47.0,97.672131,127.0,81.0,61.827869,57.0,100.0,82.387097,WHITE,M,57.438861,2


In [24]:
train_set.columns

Index(['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'r

In [37]:
x = train_set.loc[:, ['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'race', 'gender',
       'admission_age']]
y = train_set['aki']

In [40]:
x.columns

Index(['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'r

In [51]:
list(reversed(['ptt_max', 'ptt_min', 'inr_min', 'inr_max', 'pt_min', 'pt_max',
       'calcium_max.1', 'calcium_min.1', 'temperature_max', 'temperature_min',
       'temperature_mean', 'glucose_min', 'glucose_max', 'glucose_mean',
       'weight_admit', 'glucose_max.2', 'glucose_min.2', 'aniongap_max',
       'aniongap_min', 'gcs_motor', 'potassium_min.1', 'potassium_max.1',
       'bicarbonate_max.1', 'bicarbonate_min.1', 'sodium_min.1',
       'sodium_max.1', 'chloride_min.1', 'chloride_max.1', 'gcs_verbal',
       'hemoglobin_max.1', 'hemoglobin_min.1', 'wbc_max', 'wbc_min',
       'platelets_max', 'platelets_min', 'hematocrit_max.1',
       'hematocrit_min.1', 'bun_max', 'bun_min', 'gcs_eyes', 'gcs_min',
       'gcs_unable', 'resp_rate_mean', 'resp_rate_max', 'resp_rate_min',
       'spo2_max', 'spo2_mean', 'spo2_min', 'mbp_min', 'mbp_max', 'mbp_mean',
       'dbp_max', 'dbp_min', 'sbp_mean', 'sbp_max', 'sbp_min', 'dbp_mean',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'race', 'gender',
       'admission_age']))

['admission_age',
 'gender',
 'race',
 'heart_rate_mean',
 'heart_rate_max',
 'heart_rate_min',
 'dbp_mean',
 'sbp_min',
 'sbp_max',
 'sbp_mean',
 'dbp_min',
 'dbp_max',
 'mbp_mean',
 'mbp_max',
 'mbp_min',
 'spo2_min',
 'spo2_mean',
 'spo2_max',
 'resp_rate_min',
 'resp_rate_max',
 'resp_rate_mean',
 'gcs_unable',
 'gcs_min',
 'gcs_eyes',
 'bun_min',
 'bun_max',
 'hematocrit_min.1',
 'hematocrit_max.1',
 'platelets_min',
 'platelets_max',
 'wbc_min',
 'wbc_max',
 'hemoglobin_min.1',
 'hemoglobin_max.1',
 'gcs_verbal',
 'chloride_max.1',
 'chloride_min.1',
 'sodium_max.1',
 'sodium_min.1',
 'bicarbonate_min.1',
 'bicarbonate_max.1',
 'potassium_max.1',
 'potassium_min.1',
 'gcs_motor',
 'aniongap_min',
 'aniongap_max',
 'glucose_min.2',
 'glucose_max.2',
 'weight_admit',
 'glucose_mean',
 'glucose_max',
 'glucose_min',
 'temperature_mean',
 'temperature_min',
 'temperature_max',
 'calcium_min.1',
 'calcium_max.1',
 'pt_max',
 'pt_min',
 'inr_max',
 'inr_min',
 'ptt_min',
 'ptt_max']

In [59]:

x_missing_reg = x.copy()
sortindex = ['admission_age','gender','race','heart_rate_mean','heart_rate_max','heart_rate_min','dbp_mean','sbp_min','sbp_max','sbp_mean','dbp_min','dbp_max','mbp_mean','mbp_max','mbp_min','spo2_min','spo2_mean','spo2_max','resp_rate_min','resp_rate_max','resp_rate_mean','gcs_unable','gcs_min','gcs_eyes','bun_min','bun_max','hematocrit_min.1','hematocrit_max.1','platelets_min','platelets_max','wbc_min','wbc_max','hemoglobin_min.1','hemoglobin_max.1','gcs_verbal','chloride_max.1','chloride_min.1','sodium_max.1','sodium_min.1','bicarbonate_min.1','bicarbonate_max.1','potassium_max.1','potassium_min.1','gcs_motor','aniongap_min','aniongap_max','glucose_min.2','glucose_max.2','weight_admit','glucose_mean','glucose_max','glucose_min','temperature_mean','temperature_min','temperature_max','calcium_min.1','calcium_max.1','pt_max','pt_min','inr_max','inr_min','ptt_min','ptt_max']
# sortindex = sortindex[:-3]

for i in sortindex:
    df = x_missing_reg
    fillc = df.loc[: , i]
    df = pd.concat([df.loc[: , df.columns != i] , pd.DataFrame(y)] , axis=1)

    df_0 = SimpleImputer(missing_values=np.nan , strategy="constant" , fill_value=0).fit_transform(df)
    df_0 = pd.DataFrame(df_0)
    df_0.index = df.index
	
    y_train = fillc[fillc.notnull()]
    y_test = fillc[fillc.isnull()]
    x_train = df_0.loc[y_train.index , :]
    x_test = df_0.loc[y_test.index , : ]

    if x_test.shape[0] == 0:
        print(f'特征 {i} 无缺失值')
        continue

    rfc = RandomForestRegressor()
    rfc.fit(x_train , y_train)
    y_predict = rfc.predict(x_test)

    x_missing_reg.loc[x_missing_reg.loc[:,i].isnull() , i] = y_predict
    print(f'特征 {i} 处理完毕，缺失值还有 {x_missing_reg[i].isnull().sum(axis=0)} 个')


特征 admission_age 无缺失值
特征 gender 无缺失值
特征 race 无缺失值
特征 heart_rate_mean 处理完毕，缺失值还有 0 个
特征 heart_rate_max 处理完毕，缺失值还有 0 个
特征 heart_rate_min 处理完毕，缺失值还有 0 个
特征 dbp_mean 处理完毕，缺失值还有 0 个
特征 sbp_min 处理完毕，缺失值还有 0 个
特征 sbp_max 处理完毕，缺失值还有 0 个
特征 sbp_mean 处理完毕，缺失值还有 0 个
特征 dbp_min 处理完毕，缺失值还有 0 个
特征 dbp_max 处理完毕，缺失值还有 0 个
特征 mbp_mean 处理完毕，缺失值还有 0 个
特征 mbp_max 处理完毕，缺失值还有 0 个
特征 mbp_min 处理完毕，缺失值还有 0 个
特征 spo2_min 处理完毕，缺失值还有 0 个
特征 spo2_mean 处理完毕，缺失值还有 0 个
特征 spo2_max 处理完毕，缺失值还有 0 个
特征 resp_rate_min 处理完毕，缺失值还有 0 个
特征 resp_rate_max 处理完毕，缺失值还有 0 个
特征 resp_rate_mean 处理完毕，缺失值还有 0 个
特征 gcs_unable 处理完毕，缺失值还有 0 个
特征 gcs_min 处理完毕，缺失值还有 0 个
特征 gcs_eyes 处理完毕，缺失值还有 0 个
特征 bun_min 处理完毕，缺失值还有 0 个
特征 bun_max 处理完毕，缺失值还有 0 个
特征 hematocrit_min.1 处理完毕，缺失值还有 0 个
特征 hematocrit_max.1 处理完毕，缺失值还有 0 个
特征 platelets_min 处理完毕，缺失值还有 0 个
特征 platelets_max 处理完毕，缺失值还有 0 个
特征 wbc_min 处理完毕，缺失值还有 0 个
特征 wbc_max 处理完毕，缺失值还有 0 个
特征 hemoglobin_min.1 处理完毕，缺失值还有 0 个
特征 hemoglobin_max.1 处理完毕，缺失值还有 0 个
特征 gcs_verbal 处理完毕，缺失值还有 0 个
特征 chloride_max

In [60]:
x_missing_reg.shape

(48374, 63)

In [61]:
print(x_missing_reg.isnull().sum(axis=0))


ptt_max              0
ptt_min              0
inr_min              0
inr_max              0
pt_min               0
pt_max               0
calcium_max.1        0
calcium_min.1        0
temperature_max      0
temperature_min      0
temperature_mean     0
glucose_min          0
glucose_max          0
glucose_mean         0
weight_admit         0
glucose_max.2        0
glucose_min.2        0
aniongap_max         0
aniongap_min         0
gcs_motor            0
potassium_min.1      0
potassium_max.1      0
bicarbonate_max.1    0
bicarbonate_min.1    0
sodium_min.1         0
sodium_max.1         0
chloride_min.1       0
chloride_max.1       0
gcs_verbal           0
hemoglobin_max.1     0
hemoglobin_min.1     0
wbc_max              0
wbc_min              0
platelets_max        0
platelets_min        0
hematocrit_max.1     0
hematocrit_min.1     0
bun_max              0
bun_min              0
gcs_eyes             0
gcs_min              0
gcs_unable           0
resp_rate_mean       0
resp_rate_m

In [63]:
train = pd.concat([x_missing_reg, y] , axis=1)

In [64]:
train.shape

(48374, 64)

In [65]:
train.head()

,ptt_max,ptt_min,inr_min,inr_max,pt_min,pt_max,calcium_max.1,calcium_min.1,temperature_max,temperature_min,temperature_mean,glucose_min,glucose_max,glucose_mean,weight_admit,glucose_max.2,glucose_min.2,aniongap_max,aniongap_min,gcs_motor,potassium_min.1,potassium_max.1,bicarbonate_max.1,bicarbonate_min.1,sodium_min.1,sodium_max.1,chloride_min.1,chloride_max.1,gcs_verbal,hemoglobin_max.1,hemoglobin_min.1,wbc_max,wbc_min,platelets_max,platelets_min,hematocrit_max.1,hematocrit_min.1,bun_max,bun_min,gcs_eyes,gcs_min,gcs_unable,resp_rate_mean,resp_rate_max,resp_rate_min,spo2_max,spo2_mean,spo2_min,mbp_min,mbp_max,mbp_mean,dbp_max,dbp_min,sbp_mean,sbp_max,sbp_min,dbp_mean,heart_rate_min,heart_rate_max,heart_rate_mean,race,gender,admission_age,aki
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36570066,29.5,28.6,1.1,1.1,11.9,12.0,9.0,8.1,36.39,35.89,36.222000,76.0,211.0,146.2,110.0,167.0,76.0,20.0,15.0,6.00,4.8,6.7,28.0,23.0,131.0,138.0,93.0,101.0,5.0,9.8,9.2,14.3,11.0,333.0,304.0,29.2,27.8,52.0,47.0,4.0,15.0,0.0,16.833333,24.0,13.0,100.0,98.916667,96.0,56.0,69.0,63.090909,58.0,40.0,116.136364,126.0,103.0,47.863636,96.0,104.0,100.083333,7,0,79.953141,3
39307659,150.0,28.4,1.9,2.3,20.0,24.7,8.4,7.9,37.00,36.28,36.558000,127.0,132.0,129.5,82.0,143.0,127.0,22.0,13.0,6.00,2.9,4.5,29.0,19.0,126.0,132.0,89.0,93.0,5.0,13.0,12.1,14.9,12.0,247.0,199.0,39.5,36.3,30.0,23.0,4.0,15.0,0.0,29.611111,38.0,17.0,98.0,95.444444,91.0,65.0,96.0,79.555556,89.0,56.0,109.833333,127.0,97.0,70.166667,72.0,134.0,97.263158,32,0,78.194169,0
38743306,26.0,26.0,1.1,1.1,12.1,12.1,8.7,8.7,37.00,36.67,36.805000,207.0,305.0,254.0,62.1,266.0,266.0,12.0,12.0,6.00,3.8,3.8,27.0,27.0,137.0,137.0,102.0,102.0,5.0,9.5,9.5,7.5,7.5,346.0,346.0,31.1,31.1,8.0,8.0,4.0,15.0,0.0,16.000000,19.0,14.0,100.0,97.166667,94.0,69.0,108.0,85.692308,99.0,56.0,112.153846,143.0,95.0,73.307692,60.0,97.0,84.166667,28,0,65.602396,2
32339865,26.1,26.1,1.2,1.2,12.8,12.8,9.5,8.3,37.28,36.83,37.087143,111.0,120.0,117.0,113.1,136.0,120.0,11.0,10.0,1.00,4.3,4.5,39.0,36.0,140.0,141.0,92.0,93.0,0.0,18.6,16.1,7.2,5.1,178.0,146.0,62.8,56.5,39.0,39.0,1.0,15.0,1.0,22.518519,31.0,14.0,100.0,94.800000,89.0,71.0,109.0,96.720000,94.0,60.0,138.160000,150.0,113.0,80.200000,59.0,87.0,71.461538,27,0,64.906629,2
35526987,39.4,27.7,1.0,1.1,10.4,11.9,8.0,7.5,38.33,36.78,37.158750,158.0,213.0,187.0,97.4,196.0,132.0,16.0,13.0,1.61,4.2,4.5,22.0,18.0,137.0,137.0,103.0,109.0,0.0,17.1,14.0,21.1,14.6,208.0,147.0,54.3,44.0,23.0,22.0,1.0,15.0,1.0,24.230159,32.0,18.0,99.0,95.951613,90.0,56.0,100.0,70.475806,95.0,47.0,97.672131,127.0,81.0,61.827869,57.0,100.0,82.387097,28,1,57.438861,2


In [66]:
train['gender'].value_counts()

1    27046
0    21328
Name: gender, dtype: int64

In [67]:
train['race'].value_counts()

28    31017
27     5293
7      3647
22     1653
31      865
26      693
1       588
3       519
18      503
10      476
32      410
23      358
14      328
8       298
9       262
6       181
5       158
24      151
2       117
30      111
29       98
15       96
0        85
21       77
19       68
17       50
20       49
25       45
4        45
12       38
13       37
16       31
11       27
Name: race, dtype: int64

In [77]:
# train["race"] = sph.loc[train.index, "race"]
# train["gender"] = sph.loc[train.index, "gender"]
# test_set["race"] = sph.loc[test_set.index, "race"]
# test_set["gender"] = sph.loc[test_set.index, "gender"]

In [79]:
train.to_csv('./data/train.csv')
test_set.to_csv('./data/test.csv')